In [2]:
from bs4 import BeautifulSoup
import requests
import time



In [3]:
base_link = "https://krisha.kz/"

In [4]:
def get_info_from_header(text):
    info = text.split("·")
    main_info = [info[0].strip()[0], info[len(info) - 1].split(",")[1].strip()]
    return main_info
def get_info_from_price(price):
    price_text = price.strip()
    price_text = price_text.replace("&nbsp;", "")
    price_text = price_text.replace("\xa0", "")
    return price_text[:len(price_text) - 11] + "00"

def get_first_int(s):
    f = 0
    mult = 10
    for i in s:
        if i.isnumeric():
            f *= 10
            f += int(i)
        else:
            break
    return f
        
        

In [5]:
cell_to_scrape = requests.get(f"https://krisha.kz/a/show/690228805")
soup_cell = BeautifulSoup(cell_to_scrape.text, "html.parser")

a = soup_cell.findAll("dl")

In [6]:
a[0].dt.attrs

{'data-name': 'flat.priv_dorm'}

In [10]:
from bs4 import BeautifulSoup
import requests
import time

def get_info_from_header(text):
    info = text.split("·")
    main_info = [info[0].strip()[0], info[-1].split(",")[1].strip()]
    return main_info

def get_info_from_price(price):
    price_text = price.strip().replace("&nbsp;", "").replace("\xa0", "")
    return price_text[:-11] + "00"

def get_first_int(s):
    result = ''.join(filter(str.isdigit, s))
    return int(result) if result else 0

def extract_data(soup_cell, data_name, default="No Information"):
    element = soup_cell.find("div", attrs={"data-name": data_name})
    if element:
        return element.findAll("div")[2].text.strip()
    return default

def extract_from_dl(soup_cell, data_name, default="No Information"):
    dl = soup_cell.findAll("dl")
    for d in dl:
        if d.dt.attrs.get('data-name') == data_name:
            return d.dd.text.strip()
    return default

list_of_apartments = []
start_time = time.time()

for p in range(1, 2):
    page_to_scrape = requests.get(f"https://krisha.kz/arenda/kvartiry/almaty/?rent-period-switch=%2Farenda%2Fkvartiry&page={p}")
    soup = BeautifulSoup(page_to_scrape.text, "html.parser")
    links = soup.findAll("a", attrs={"class": "a-card__title"})

    for l in links:
        cell_to_scrape = requests.get(f"https://krisha.kz{l.get('href')}")
        soup_cell = BeautifulSoup(cell_to_scrape.text, "html.parser")

        header = soup_cell.find("h1")
        cell_main_info = get_info_from_header(header.text)

        price_html = soup_cell.find("div", attrs={"class": "offer__price"})
        price_info = get_info_from_price(price_html.text)

        city_html = soup_cell.find("div", attrs={"class": "offer__location offer__advert-short-info"})
        city_info = city_html.find('span').text.strip()

        floor_default = extract_data(soup_cell, "flat.floor", default=0)
        area_default = extract_data(soup_cell, "live.square", default=0)
        kitchen_studio_default = extract_data(soup_cell, "kitchen_studio")
        rent_renovation_default = extract_data(soup_cell, "flat.rent_renovation", 
                                               default=extract_data(soup_cell, "flat.renovation"))
        furniture_default = extract_data(soup_cell, "live.furniture")
        priv_dorm_default = extract_data(soup_cell, "flat.priv_dorm", 
                                         default=extract_from_dl(soup_cell, "flat.priv_dorm"))
        security_default = extract_data(soup_cell, "flat.security", 
                                        default=extract_from_dl(soup_cell, "flat.security"))
        furniture_detailed_default = extract_data(soup_cell, "flat.furniture", 
                                                  default=extract_from_dl(soup_cell, "flat.furniture"))
        facilities_default = extract_data(soup_cell, "flat.facilities", 
                                          default=extract_from_dl(soup_cell, "flat.facilities"))
        bathroom_default = extract_data(soup_cell, "bathroom", 
                                        default=extract_from_dl(soup_cell, "bathroom"))
        window_default = extract_data(soup_cell, "window_side", 
                                      default=extract_from_dl(soup_cell, "window_side"))
        balcony_default = extract_data(soup_cell, "balcony_count", 
                                       default=extract_from_dl(soup_cell, "balcony_count"))
        separated_toilet_default = extract_data(soup_cell, "separated_toilet", 
                                                default=extract_from_dl(soup_cell, "separated_toilet"))
        zhiloi_complex_default = extract_data(soup_cell, "map.complex")
        house_year_default = extract_data(soup_cell, "house.year")
        parking_default = extract_data(soup_cell, "flat.parking")
        toilet_count_default = extract_data(soup_cell, "toilet_count", 
                                            default=extract_from_dl(soup_cell, "toilet_count"))
        loggia_count_default = extract_data(soup_cell, "loggia_count", 
                                            default=extract_from_dl(soup_cell, "loggia_count"))
        people_default = extract_data(soup_cell, "who_match", 
                                      default=extract_from_dl(soup_cell, "who_match"))
        balcony_glass_default = extract_data(soup_cell, "flat.balcony_g", 
                                             default=extract_from_dl(soup_cell, "flat.balcony_g"))

        description_html = soup_cell.find("div", attrs={"class": "js-description a-text a-text-white-spaces"})
        description_default = description_html.text.strip() if description_html else "No Information"

        apartment = {
            "url": f"https://krisha.kz{l.get('href')}",
            "rooms": cell_main_info[0],
            "adress": cell_main_info[1],
            "price": price_info,
            "city_region": city_info,
            "floor": floor_default,
            "area": area_default,
            "kitchen_studio": kitchen_studio_default,
            "apartment_condition": rent_renovation_default,
            "is_apartment_has_furniture(full or no)": furniture_default,
            "is_previously_dormitory": priv_dorm_default,
            "security": security_default,
            "furniture_detailed": furniture_detailed_default,
            "facilities": facilities_default,
            "bathroom": bathroom_default,
            "zhiloi_complex": zhiloi_complex_default,
            "house_year": house_year_default,
            "parking": parking_default,
            "is_separated_toilet": separated_toilet_default,
            "toilet_count": toilet_count_default,
            "window_side": window_default,
            "balcony_count": balcony_default,
            "loggia_count": loggia_count_default,
            "people_that_match": people_default,
            "description": description_default
        }
        list_of_apartments.append(apartment)

print("--- %s seconds ---" % (time.time() - start_time))

--- 12.432145833969116 seconds ---


In [11]:
len(list_of_apartments)

23

In [12]:
len(list_of_apartments[0])

25

In [13]:
list_of_apartments

[{'url': 'https://krisha.kz/a/show/761058846?srchid=019666fc-c084-7cf9-80d9-9b1eb00535fc&srchtype=hot_block_filter&srchpos=1',
  'rooms': '2',
  'adress': 'Розыбакиева 388/Б2',
  'price': '420000',
  'city_region': 'Алматы, Бостандыкский р-н',
  'floor': '15 из 16',
  'area': '65 м²',
  'kitchen_studio': 'да',
  'apartment_condition': 'No Information',
  'is_apartment_has_furniture(full or no)': 'полностью',
  'is_previously_dormitory': 'нет',
  'security': 'No Information',
  'furniture_detailed': 'No Information',
  'facilities': 'No Information',
  'bathroom': 'No Information',
  'zhiloi_complex': 'ATRIA',
  'house_year': 'No Information',
  'parking': 'No Information',
  'is_separated_toilet': 'совмещен',
  'toilet_count': '1',
  'window_side': 'на улицу',
  'balcony_count': 'No Information',
  'loggia_count': '1',
  'people_that_match': 'No Information',
  'description': 'Сдаем квартиру в Жилом комплексе ATRIA, бизнес Класса, расположеном в одном из самых притягательных и динамичн

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(list_of_apartments)

In [16]:
df

,url,rooms,adress,price,city_region,floor,area,kitchen_studio,apartment_condition,is_apartment_has_furniture(full or no),...,zhiloi_complex,house_year,parking,is_separated_toilet,toilet_count,window_side,balcony_count,loggia_count,people_that_match,description
0,https://krisha.kz/a/show/761058846?srchid=0196...,2,Розыбакиева 388/Б2,420000,"Алматы, Бостандыкский р-н",15 из 16,65 м²,да,No Information,полностью,...,ATRIA,No Information,No Information,совмещен,1,на улицу,No Information,1,No Information,"Сдаем квартиру в Жилом комплексе ATRIA, бизнес..."
1,https://krisha.kz/a/show/692949126?srchid=0196...,4,Макатаева 2,1200000,"Алматы, Медеуский р-н",15 из 17,140 м²,No Information,"свежий ремонт, новая мебель",полностью,...,Chocolate,No Information,No Information,No Information,3,No Information,No Information,No Information,No Information,Срочно сдаем в Аренду на длительный срок 4-х к...
2,https://krisha.kz/a/show/690465839?srchid=0196...,2,Жумалиева 153 — Байзакова-Джамбула,300000,"Алматы, Алмалинский р-н",8 из 12,"50 м², жилая — 32 м²",да,"не новый, но аккуратный и чистый ремонт",полностью,...,Отау,No Information,No Information,совмещен,1,во двор,1,No Information,"семейной паре, одному человеку, можно с детьми",No Information
3,https://krisha.kz/a/show/687939327?srchid=0196...,3,ул. Коктем 1 — Алатауская трасса район аэропор...,200000,Алматы,1 из 3,"52 м², Площадь кухни — 12 м²",No Information,"свежий ремонт, новая мебель",полностью,...,No Information,No Information,No Information,совмещен,1,"во двор, на улицу",нет,нет,"семейной паре, некурящим, одному человеку","Срочно сдам на длительный срок 3ком. кв, 200ты..."
4,https://krisha.kz/a/show/1000805404?srchid=019...,2,Аль-Фараби,550000,"Алматы, Бостандыкский р-н",7 из 19,60 м²,да,"свежий ремонт, новая мебель",полностью,...,Metropole,No Information,No Information,"разделен, совмещен",1,во двор,нет,No Information,"семейной паре, некурящим, одному человеку, мож...","Современная, светлая квартира 80 кв. м с дизай..."
5,https://krisha.kz/a/show/1000804486?srchid=019...,3,Аль-Фараби — Сейфуллина,800000,"Алматы, Бостандыкский р-н",10 из 21,90 м²,да,"свежий ремонт, новая мебель",полностью,...,AL’FARABI 27,No Information,No Information,совмещен,1,"во двор, на улицу",No Information,No Information,"семейной паре, некурящим, одному человеку, мож...",Новая роскошная квартира с дизайнерским ремонт...
6,https://krisha.kz/a/show/1000803471?srchid=019...,2,Аль-Фараби — Сейфуллина,600000,"Алматы, Бостандыкский р-н",8 из 20,62 м²,да,"свежий ремонт, новая мебель",No Information,...,AL’FARABI 27,No Information,No Information,совмещен,1,"во двор, на улицу",1,No Information,"семейной паре, некурящим, одному человеку, мож...",Сдаётся 2-комнатная квартира с современным рем...
7,https://krisha.kz/a/show/696604015?srchid=0196...,1,Северное кольцо 92/2,190000,"Алматы, Алатауский р-н",1 из 9,37 м²,No Information,"свежий ремонт, новая мебель",полностью,...,Ozhet Plus,No Information,No Information,совмещен,1,во двор,нет,нет,"семейной паре, некурящим, одному человеку","Квартира новая, с мебелью, холодильник, стирал..."
8,https://krisha.kz/a/show/682713823?srchid=0196...,1,мкр Жас Канат,280000,"Алматы, Турксибский р-н",6,40 м²,No Information,"свежий ремонт, новая мебель",полностью,...,Suly’ Tan,No Information,No Information,совмещен,1,во двор,1,1,"некурящим, одному человеку",No Information
9,https://krisha.kz/a/show/671595619?srchid=0196...,1,мкр Шугыла,160000,"Алматы, Наурызбайский р-н",9 из 9,"35 м², жилая — 18 м², Площадь кухни — 10 м²",No Information,свежий ремонт,No Information,...,Nurly Dala,2020,No Information,совмещен,1,во двор,1,No Information,"семейной паре, некурящим, одному человеку","1 комн кв-ра в новом ЖК ""Нурлы дала"", напротив..."


In [17]:
df.to_csv("apartments.csv")

In [39]:
from openai import OpenAI
import json

client = OpenAI(api_key="sk-74b87290351b47acacfc94680907ed09", base_url="https://api.deepseek.com")

def get_real_estate_details(client_prompt):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a professional realtor." +
             "Extract the following real estate features from the user's input and make sure values are in Russian and return them as a Python dictionary:\n\n" +
                "{ 'rooms': value, 'adress': value, 'price': value, 'city_region': value, 'floor': value, 'area': value, 'kitchen_studio': value," + 
                " 'apartment_condition': value, 'furniture_confition': value, 'is_previously_dormitory': value, 'security': value," + 
                " 'furniture_detailed': value, 'facilities': value, 'bathroom': value, 'zhiloi_complex': value, 'house_year': value," + 
                " 'parking': value, 'is_separated_toilet': value, 'toilet_count': value, 'description': value }\n\n" + 
                "If any information is missing, replace it with 'No Information'. Ensure the response is a properly formatted JSON string."},
            {"role": "user", "content": client_prompt},
        ],
        stream=False
    )
    return response.choices[0].message.content

# Example usage
client_input = "Ищу 2-комнатную квартиру в аренду в ЖК PRIMAVERA или рядом с Ходжановым, полностью меблированную, с новой техникой, окна на юго-запад, вид на горы и город, без шума от Аль-Фараби, с подземным паркингом, рядом с Ботаническим садом, на длительный срок, готов внести депозит."
formatted_response = get_real_estate_details(client_input)
print(formatted_response)


```json
{
  "rooms": "2",
  "adress": "ЖК PRIMAVERA или рядом с Ходжановым",
  "price": "No Information",
  "city_region": "No Information",
  "floor": "No Information",
  "area": "No Information",
  "kitchen_studio": "No Information",
  "apartment_condition": "No Information",
  "furniture_confition": "Полностью меблированная",
  "is_previously_dormitory": "No Information",
  "security": "No Information",
  "furniture_detailed": "Новая техника",
  "facilities": "Окна на юго-запад, вид на горы и город, без шума от Аль-Фараби, рядом с Ботаническим садом",
  "bathroom": "No Information",
  "zhiloi_complex": "ЖК PRIMAVERA",
  "house_year": "No Information",
  "parking": "Подземный паркинг",
  "is_separated_toilet": "No Information",
  "toilet_count": "No Information",
  "description": "Ищу на длительный срок, готов внести депозит."
}
```
